# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [331]:
import nltk
import numpy as np

from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from collections import Counter
from nltk.tokenize import sent_tokenize
from nltk.tokenize import sent_tokenize


In [32]:
news = open('lenta.txt', encoding='utf-8').read()

In [124]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


In [125]:
#применяем нормализацию к текстам
norm_news = normalize(news)

#cоздаем частотные словари
vocab_news = Counter(norm_news)

In [126]:
vocab_news.most_common(10)

[('в', 72412),
 ('и', 33290),
 ('на', 28434),
 ('по', 19490),
 ('что', 17031),
 ('с', 15921),
 ('не', 12702),
 ('из', 7727),
 ('о', 7515),
 ('как', 7514)]

In [127]:
#переводим абсолютные частоты в вероятности, разделив количество употреблений слова на общее число слов в корпусе
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(10)

[('в', 0.04808907489694771),
 ('и', 0.0221080111489724),
 ('на', 0.018883123731146926),
 ('по', 0.012943380513471676),
 ('что', 0.011310349590812525),
 ('с', 0.01057319451795703),
 ('не', 0.008435444806676101),
 ('из', 0.005131529052211166),
 ('о', 0.00499073907433246),
 ('как', 0.0049900749706632205)]

In [322]:
sentences_news = [['<start>' + ""] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [323]:
# берем отрывок из 50 предложений для расчета перплексии

test_news = sentences_news[-50:]

# убираем эти предложения из основной выборки

sentences_news = sentences_news[:-50]


In [324]:
# ищем кол-во вхождений каждой n-граммы и проходимся по токенам
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [325]:
#Создаем частотные словари униграм, биграм и триграм

unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

In [326]:
trigrams_news.most_common(10)

[('<start> об этом', 1578),
 ('<start> по словам', 1549),
 ('сообщает риа новости', 1324),
 ('со ссылкой на', 1242),
 ('риа новости <end>', 1228),
 ('<start> кроме того', 1070),
 ('<start> как сообщает', 1064),
 ('<start> напомним что', 1005),
 ('по его словам', 899),
 ('<start> по его', 868)]

In [327]:
bigrams_news.most_common(10)

[('<start> в', 7960),
 ('<start> по', 6210),
 ('<start> как', 3736),
 ('риа новости', 3502),
 ('по словам', 1971),
 ('об этом', 1794),
 ('<start> однако', 1693),
 ('<start> на', 1642),
 ('что в', 1621),
 ('<start> об', 1618)]

### Генерация текста


In [332]:
from scipy.sparse import lil_matrix

In [ ]:
#создаем матрицу вероятностей для новостей
matrix_news = lil_matrix((len(bigrams_news), 
                   len(unigrams_news)))
id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    w1, w2, w3 = ngram.split()
    bigram = w1 + ' ' + w2
    trigram = w1 + ' ' + w2 + ' ' + w3
    matrix_news[bigram2id_news[bigram], word2id_news[word3]] =  (trigrams_news[trigram]/
                                                                bigrams_news[bigram])



In [ ]:
def generate(matrix, id2word, bigram2id, id2bigram, n=250, start='<start>'):
    text = []
    current_idx = id2bigram[start]

    
    for i in range(n):
        
        chosen = matrix[current_idx].argmax()
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = bigram2id[start]
            
        else:
            bigram = id2bigram[current_idx]
            trigram = bigram + ' ' + id2word[chosen]
            w1, w2, w3 = trigram.split()
            new_bigram = w2 + ' ' + w3
            chosen = bigram2id[new_bigram]
   
        
        current_idx = chosen

    return ' '.join(text)

In [ ]:
print(generate(matrix_news, id2word_news, id2bigram_news, bigram2id_news).replace('<end>', '\n'))

### Перплексия

In [248]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

In [253]:
perplexities = []

for sentence in test_news[:15]:
    prob = []

    
    for ngram in ngrammer(sentence, n=3):
        word1, word2, word3 = ngram.split()
        bigram = word1 + ' ' + word2
        
        if gram2 in bigrams_news and ngram in trigrams_news:        
            prob.append(np.log(trigrams_news[ngram] / bigrams_news[gram2]))
        else:
            prob.append(np.log(0.00001))
    
    perplexities.append(perplexity(prob))

In [254]:
perplexities

[1476.009465923114,
 10307.66269203421,
 27045.318696275972,
 99999.99999999991,
 23134.59701913287,
 17443.65142811791,
 30493.357264575407,
 16614.712259318607,
 12956.68420148759,
 8994.822584971907,
 143.8539784980612,
 99999.99999999991,
 425.0215501554316,
 183.14458224906917,
 20670.11216429356]

In [255]:
np.mean(perplexities)  ##усредненная перплексия

24659.263192468898

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

2. Что такое сглаживание (smoothing)?